# Pretrained Model, regression (target MEM-Score), Elastic Net


In [1]:
import os
import pandas as pd
import numpy as np
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/Colab Notebooks/Innolab/Github') 

Mounted at /content/drive


In [2]:
from src.preprocessing.preprocessing_matlab_files import preprocess_mat_files
from src.preprocessing.data_preparation import prepare_data

In [3]:
matlab_dir = "/content/drive/MyDrive/Colab Notebooks/Innolab/Matlab Files" # Enter the directory for the matlab files
excel_path = "/content/drive/MyDrive/Colab Notebooks/Innolab/DELCODE_dataset.xlsx" # Enter the directory for the corresponding excel sheet
write_dir = "/content/drive/MyDrive/Colab Notebooks/Innolab/write" # ...
export_file = False # rename to export file
preprocessing_type = 'aggregation' 

df = preprocess_mat_files(matlab_dir = matlab_dir, excel_path = excel_path, write_dir = write_dir,
                           export_file = export_file, preprocessing_type = preprocessing_type, statistic = 'max')

loading files
Starting Preprocessing
Creating Final Dataset
Done!


In [4]:
df

,ConnID,Repseudonym,siteid,age,visdat,sex,prmdiag,edyears,MEM_score,Apoe,IDs,0_0,0_1,0_2,0_3,0_4,0_5,0_6,0_7,1_1,1_2,1_3,1_4,1_5,1_6,1_7,2_2,2_3,2_4,2_5,2_6,2_7,3_3,3_4,3_5,3_6,3_7,4_4,4_5,4_6,4_7,5_5,5_6,5_7,6_6,6_7,7_7
0,1,0a8d02f2b,11,66,17.08.2016,0,2,17,0.054016,0.0,1.0,1.442019,0.518327,0.768837,0.781428,0.798942,0.713009,0.569728,0.693034,1.493739,1.055437,1.217551,0.83579,0.699596,1.090191,0.921686,1.499958,1.446869,1.173337,0.854638,1.044583,1.215168,1.374804,0.932106,0.708553,1.14529,1.064851,1.162719,0.580244,1.430655,1.042193,1.244207,0.820703,0.885204,1.26936,1.249158,1.370669
1,2,0a71a953d,17,72,30.03.2015,0,1,20,-0.468749,1.0,2.0,1.052224,0.695097,0.608619,0.617755,0.78139,0.700752,0.661261,0.716709,1.791651,1.182725,1.42357,0.831551,0.855352,1.033703,1.231956,1.824582,1.567818,1.511543,0.75816,1.150066,0.907262,1.392219,1.032674,0.913651,1.212448,1.11951,1.315684,0.75934,0.923772,0.930864,1.534952,0.690841,0.953771,1.211397,1.204627,1.371878
2,3,0a61339db,11,72,13.05.2015,1,1,16,-0.093521,0.0,3.0,0.849411,0.564845,0.555714,0.461625,0.495856,0.788707,0.573609,0.586259,0.976541,0.613206,0.851963,0.370562,0.57159,0.787161,1.130506,1.008474,1.047959,0.770432,0.493032,0.782779,0.576682,1.325337,1.001475,0.600156,0.875494,1.110678,1.021635,0.491159,0.861457,0.908652,0.917063,0.923844,0.699996,0.932172,1.02339,1.326019
3,4,0b28aed58,17,76,18.01.2016,0,1,20,0.466027,0.0,4.0,1.108932,0.911937,0.672048,0.726071,0.781558,0.692417,0.640137,0.889349,1.832284,0.747189,1.01545,0.591824,0.752035,0.702183,1.012694,1.641574,1.338708,1.412591,0.839167,1.014339,1.173346,1.393465,1.042414,0.605472,1.01288,1.523996,1.277929,0.684653,1.290996,1.010103,1.071911,0.774992,0.901711,0.969546,1.222428,1.318915
4,5,0c1c5ae77,8,64,12.03.2015,1,1,13,1.452099,0.0,5.0,0.902319,0.762414,0.702308,0.746809,0.942794,0.745581,0.647355,0.882981,1.391585,0.816195,0.855742,0.694862,0.70731,0.793479,1.369496,1.184785,1.120838,1.121519,0.565561,0.868133,0.722518,1.071553,0.803008,0.689007,0.988444,0.808636,1.108088,0.713629,1.128626,0.83524,1.140538,0.879643,0.923432,1.153777,0.889666,1.244347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,906,f816d269a,16,84,2017-11-21,2,3,18,-2.388095,0.0,906.0,1.417167,0.612684,0.789715,0.470563,0.879898,0.911326,0.945266,0.965765,1.673786,0.815731,1.171023,0.80011,0.492837,0.885967,0.996097,1.460167,1.441553,1.374246,0.559355,0.923871,0.806196,1.305496,1.182395,0.56674,1.025421,0.828947,1.038595,0.71342,1.064049,0.880292,0.858801,0.996523,0.70373,0.887294,1.088491,1.377185
905,907,f98419539,5,71,2014-09-11,2,3,13,-1.860675,1.0,907.0,1.107081,0.489421,0.620626,0.396967,1.084753,0.643209,0.861978,1.207757,1.509016,0.881523,0.977685,0.640263,0.565235,0.739145,1.026662,1.510382,1.111873,1.226397,0.74193,0.906277,1.366042,1.258578,0.829937,0.639695,1.048475,0.877682,1.387487,0.621841,1.000504,1.59777,1.281182,0.760985,0.987812,1.168594,0.962629,1.500568
906,908,fade343c8,8,78,2015-11-23,2,3,8,-1.868184,1.0,908.0,0.769773,0.426069,0.621418,0.445918,0.537328,0.650858,0.611799,0.927391,0.908209,0.627183,0.848678,0.461005,0.47463,0.637235,0.649359,0.84264,0.94299,0.799259,0.518328,0.541375,0.779953,0.904409,0.608589,0.659208,0.865961,0.8663,0.836314,0.502671,0.726753,0.808973,0.914337,0.60809,0.578736,0.769957,1.071035,1.132849
907,909,fbc8a48a0,14,61,2017-03-20,2,3,9,-2.432351,0.0,909.0,0.778697,0.526224,0.526895,0.466178,0.669391,0.619619,0.61346,0.634253,1.335476,0.497276,0.845021,0.599592,0.627642,0.881996,1.127493,1.301591,1.258621,0.890437,0.537687,0.614483,0.689756,0.954526,0.812523,0.400247,1.182646,0.834384,1.158202,0.579332,0.986103,1.063459,0.906869,0.812011,0.861273,1.008168,1.016017,1.177611


In [5]:
classification = False
columns_drop = ["ConnID", "Repseudonym", "siteid", "visdat", "prmdiag", "Apoe", "IDs"]
target = "MEM_score"
y_0 = [0]
y_1 = [2, 3]
train_size = 0.8
seed = 123
split = True

In [6]:
# preparation of data
ytrain, Xtrain, ytest, Xtest = prepare_data(data = df, classification = classification,
                         columns_drop = columns_drop, target = target, y_0 = y_0, y_1 = y_1,
                         train_size = train_size, seed = seed, split = split)


pd.concat([ytest, Xtest], axis = 1) # test data

,y,age,sex,edyears,0_0,0_1,0_2,0_3,0_4,0_5,0_6,0_7,1_1,1_2,1_3,1_4,1_5,1_6,1_7,2_2,2_3,2_4,2_5,2_6,2_7,3_3,3_4,3_5,3_6,3_7,4_4,4_5,4_6,4_7,5_5,5_6,5_7,6_6,6_7,7_7
721,0.973914,-0.500490,0.690935,-0.851425,0.319398,-1.844707,0.608817,-0.762756,0.177614,-1.289737,-0.163846,0.080191,-0.293265,-0.684563,1.720462,-0.217598,0.089855,1.204993,-0.539437,-0.322370,-0.733844,0.646721,1.770103,0.011700,2.294896,0.923172,1.468505,1.271606,0.101851,0.268390,-0.144342,0.614480,0.189358,1.729321,-0.092948,2.391770,0.626931,0.073113,1.387568,0.574251
335,0.281476,-0.174161,0.690935,0.819703,0.280311,-0.067356,-1.242309,0.175594,-1.054499,1.189923,0.065493,-0.545692,-1.377978,-0.094067,1.136884,-0.539034,-0.039485,1.950072,-0.532648,-0.095614,1.000677,-0.598555,-0.253072,-0.198897,-1.808342,0.234433,0.102808,1.664556,0.813621,-0.215962,0.395161,0.260658,0.145870,0.229494,-0.264978,2.096949,0.413661,1.572232,-0.499216,-1.314849
590,0.19615,-0.337326,-1.041167,0.151252,0.307587,0.946427,0.420190,0.290190,0.132312,0.350674,1.259974,-0.120912,0.221306,1.047020,0.609311,1.626935,0.238952,-0.090743,0.635127,1.003129,1.651781,1.035723,-0.615553,-0.872159,0.663014,-0.391860,1.344107,-0.361466,1.116038,0.787469,0.616234,-0.137485,-0.709273,1.012785,0.254992,-0.900028,-0.553861,0.047601,0.655085,0.711177
7,-1.815235,0.641659,0.690935,-0.851425,5.639412,5.851921,8.295143,6.289523,5.386507,4.652472,6.242319,5.500924,0.108882,2.993161,5.253364,5.467365,5.798147,2.809245,3.612805,3.931398,3.299855,2.230250,2.793790,4.247542,5.715341,2.130052,3.170809,4.170649,1.481309,4.414429,1.363812,6.091369,2.804493,5.487018,4.702684,3.303192,4.247872,0.092975,3.507035,4.608404
567,0.676812,-0.174161,0.690935,-0.517199,-0.485269,-0.387766,-0.346384,0.159251,0.498480,-0.658385,0.070357,0.628626,-0.172459,-1.314492,-1.359774,-0.590387,-1.269186,0.427655,-0.060145,0.056625,-0.907655,-0.536743,-0.566256,-0.930629,-1.592623,0.568333,0.060242,0.802008,0.982012,1.149151,-0.746062,0.116492,1.704341,-0.471068,-0.304499,-0.397461,-0.723172,1.484416,0.542388,1.974195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,0.87983,-0.337326,-1.041167,0.151252,-0.748470,-0.637755,0.123611,-0.849856,-0.849879,-0.462632,-1.018232,-1.243678,0.168540,-0.121783,-0.020346,0.296853,-0.137515,0.590560,1.074794,-1.642624,-0.689295,0.296850,-0.124211,0.150195,-0.310498,0.064532,-1.123064,0.173164,-1.050649,-1.033468,-0.918569,0.303940,-0.184469,-0.493302,0.709378,-0.768470,1.898536,1.639827,-1.280220,0.089260
793,-0.39345,0.967987,-1.041167,-0.517199,0.059308,-0.906389,0.384614,0.033687,0.121973,-0.045082,-0.903976,-0.003377,0.851732,0.213810,0.631948,0.021941,0.874771,-0.849463,0.026926,0.551376,1.388685,3.145571,1.007662,1.326275,0.005038,-0.574258,1.241152,-0.707691,-0.118872,0.038727,0.551397,0.131756,-0.088391,0.024909,1.691134,1.828503,0.570450,-0.256357,-0.054751,0.298028
11,1.320656,-1.153146,0.690935,-0.851425,0.121853,-0.336343,-0.724594,-0.074213,-0.513454,-1.032084,-0.654235,-0.829759,-0.942241,-0.971694,0.182752,-0.261129,-0.730707,-0.208139,0.343286,1.204586,-1.089049,0.242955,-0.706170,-0.134874,0.599614,-1.088152,-1.119917,-1.347173,-0.600783,0.202373,-0.282770,-0.624310,-1.052936,0.209459,0.410277,0.373391,-0.161329,-0.741883,-0.090875,0.390924
317,0.627507,-0.174161,-1.041167,1.488154,2.904528,-0.361858,-0.632026,0.039886,-0.784001,0.537039,0.457865,-0.653529,0.901376,0.358634,0.920254,-0.665270,-1.178234,0.013653,-1.083482,0.390425,-0.152902,-0.154320,0.021262,0.653493,0.086127,-0.255897,-0.516617,0.177196,-0.096682,-0.626417,-0.565509,0.439889,0.902871,-0.823268,0.515164,0.628415,-0.010411,0.326859,-0.234685,0.291422


# Train Model

In [7]:
from src.models.pipeline_elastic_net import model_elastic_net
from sklearn.metrics import mean_squared_error, r2_score

In [8]:
model = model_elastic_net(X_train = Xtrain, y_train = ytrain, classification = False, 
                          n_alphas_linreg = 10, 
                          l1_ratios_linreg= np.linspace(0.01,  1, 11).tolist(),
                          cv_linreg = 5)               

In [9]:
predictions = model.predict(Xtest)
mean_squared_error(ytest, predictions)

0.6757374536843218

In [10]:
r2_score(ytest, predictions)

0.3098320755571884

In [11]:
model.l1_ratio_

0.109

In [12]:
model.alpha_

0.16811683109271316

# Save Model

In [13]:
import pickle

In [14]:
filename ='pretrained_elnet_regr_agg_max.sav'
pickle.dump(model, open(filename, 'wb'))

# Visualisation Feature Importance and Coeffs (evtl Ridge)

In [ ]:
# load model
model = pickle.load(open("/content/drive/MyDrive/Colab Notebooks/Innolab/pretrained_elnet_regr_agg_max.sav", 'rb'))